In [13]:
# Imports a parser from cogent
from cogent.parse.fasta import MinimalFastaParser as parse

In [5]:
# applies for the whole segment
nprocs = 10

In [4]:
# Checking out data file.
# This file was created using the QC_basic notebook.
!head data/finalQC.fasta

>S133_53
TACGTAGGGGGCAAGCGTTGTCCGGAATTATTGGGCGTAAAGGGCGCGTAGGCGGTTCGGTAAGTCACCTGTGAAACCTCTGGGCTCAACTCAGGGCCTGCAGGCGAAACTGCCGTGCTGGAGTGTGGGAGAGGTGCGTGGAATTCCCGGTGTAGCGGTGAAATGCGTAGATATCGGGAGGAACACCTGTGGCGAAAGCGGCGCACTGGACCACAACTGACGCTGAGGCGCGAAAGCTAGGGGAGCAAACAGGATTAGATACCCTGGTAGTCCTAGCCCTAAACGATGGATGCTTGGTGTGTCGGGTACCCAATCCCGACGTGCCGAAGCTAACGCGATAAGCATCCCGCCTGGGGAGTACGGTCGCAAGGCTG
>S009_54
TACGAAGGTGGCAAGCGTTATTCGGATTTACTGGGCGTACAGGGAGCGTAGGCGGTTCGGTAAGCCCTTTGTGAAATCTTCAGGCTTAACCTGGAACAGTCGGAGGGGACTGCCGGGCTAGAGGACGGGAGAGGAGCGCGGAATTCCCGGTGTAGCGGTGAAATGCGTAGAGATCGGGAGGAAGGCCGGTGGCGAAGGCGGCGCTCTGGAACGTTTCTGATGCTGAGGCTCGAAAGCGTGGGGAGCAAACAGGATTAGATACCCTGGTAGTCCACGCCTTAAACTATGAATACTAAGTGTCGGCGGGTTACCGCCGGTGCCGCAGCTAACGCATTAAGTATTCCGCCTGGGAAGTACGGCCGCAAGGTTG
>S026_79
TACGAAGGGGGCTAGCGTTGCTCGGAATCACTGGGCGTAAAGGGTGCGTAGGCGGGTCTTTAAGTCAGGGGTGAAATCCTGGAGCTCAACTCCAGAACTGCCTTTGATACTGAAGATCTTGAGTTCGGGAGAGGTGAGTGGAACTGCGAGTGTAGAGGTGAAATTCGTAGATATTCGCAAGAACACCAGTGGCGAAGGCGGCTCACTGGCCCGATACTGACGCTGAG

In [10]:
%%bash
# Running the seq separation on mothur instead

mothur "#unique.seqs(fasta=data/finalQC.fasta)" > /dev/null

In [11]:
# Making a dictionary of the names files, splitting it into the first (ID) and second (commas list of all seqs in it)
# Then it counts their lengths and saves it in the dictionary

counts = {}

with open("data/finalQC.names") as f:
    for line in f:
        seedID, seqIDs = line.split("\t")
        count = len(seqIDs.split(","))
        counts[seedID] = count        

In [14]:
# Adds the counts from this dictionary to our finalQC.unique file so it looks like a usearch file with "size=XXX"

with open("data/finalQC.unique.usearch_names.fasta", "w") as f:
    for n, s in parse(open("data/finalQC.unique.fasta")):
        f.write(">%s;size=%s;\n%s\n"%(n,counts[n],s))  

In [15]:
!head data/finalQC.unique.usearch_names.fasta

>S133_53;size=1;
TACGTAGGGGGCAAGCGTTGTCCGGAATTATTGGGCGTAAAGGGCGCGTAGGCGGTTCGGTAAGTCACCTGTGAAACCTCTGGGCTCAACTCAGGGCCTGCAGGCGAAACTGCCGTGCTGGAGTGTGGGAGAGGTGCGTGGAATTCCCGGTGTAGCGGTGAAATGCGTAGATATCGGGAGGAACACCTGTGGCGAAAGCGGCGCACTGGACCACAACTGACGCTGAGGCGCGAAAGCTAGGGGAGCAAACAGGATTAGATACCCTGGTAGTCCTAGCCCTAAACGATGGATGCTTGGTGTGTCGGGTACCCAATCCCGACGTGCCGAAGCTAACGCGATAAGCATCCCGCCTGGGGAGTACGGTCGCAAGGCTG
>S009_54;size=1;
TACGAAGGTGGCAAGCGTTATTCGGATTTACTGGGCGTACAGGGAGCGTAGGCGGTTCGGTAAGCCCTTTGTGAAATCTTCAGGCTTAACCTGGAACAGTCGGAGGGGACTGCCGGGCTAGAGGACGGGAGAGGAGCGCGGAATTCCCGGTGTAGCGGTGAAATGCGTAGAGATCGGGAGGAAGGCCGGTGGCGAAGGCGGCGCTCTGGAACGTTTCTGATGCTGAGGCTCGAAAGCGTGGGGAGCAAACAGGATTAGATACCCTGGTAGTCCACGCCTTAAACTATGAATACTAAGTGTCGGCGGGTTACCGCCGGTGCCGCAGCTAACGCATTAAGTATTCCGCCTGGGAAGTACGGCCGCAAGGTTG
>S026_79;size=35877;
TACGAAGGGGGCTAGCGTTGCTCGGAATCACTGGGCGTAAAGGGTGCGTAGGCGGGTCTTTAAGTCAGGGGTGAAATCCTGGAGCTCAACTCCAGAACTGCCTTTGATACTGAAGATCTTGAGTTCGGGAGAGGTGAGTGGAACTGCGAGTGTAGAGGTGAAATTCGTAGATATTCGCAAGAACACCAGTGGCGAAGGC

In [7]:
# Chuck looking to see how many Gb this file is.
!du -h data/finalQC.fasta

3.8G	data/finalQC.fasta


In [16]:
# This runs out of memory.
!usearch -derep_fulllength data/finalQC.fasta -output data/finalQC.unique.fasta -sizeout -threads 20

usearch v7.0.1090_i86linux32, 4.0Gb RAM (132Gb total), 40 cores
(C) Copyright 2013 Robert C. Edgar, all rights reserved.
http://drive5.com/usearch

Licensed to: chuck.peperanney@gmail.com

00:10 2.0Gb  100.0% Reading data/finalQC.unique.usearch_names.fasta
00:19 2.4Gb 4697604 (4.7M) uniques, avg cluster 1.0, median 1, max 1
^C


In [17]:
# Sequences are sorted by size
# Here the size of clusters - we are excluding the singletons here
# You would change minsize to 1 if you wanted to include singletons
# Or, you know, just not do this step.
# But you should just get rid of them.
!usearch -sortbysize data/finalQC.unique.usearch_names.fasta -output data/finalQC.unique_sorted.fasta -minsize 2

usearch v7.0.1090_i86linux32, 4.0Gb RAM (132Gb total), 40 cores
(C) Copyright 2013 Robert C. Edgar, all rights reserved.
http://drive5.com/usearch

Licensed to: chuck.peperanney@gmail.com

00:10 2.0Gb  100.0% Reading data/finalQC.unique.usearch_names.fasta
00:10 2.0Gb Getting sizes                                          
00:19 2.0Gb Sorting 581709 sequences
00:22 2.0Gb  100.0% Writing data/finalQC.unique_sorted.fasta


In [18]:
# Checking data
# You can see here, the first two sequences we saw above are now gone.
!head data/finalQC.unique_sorted.fasta

>S139_760;size=37566;
TACGGAGGGGGCTAGCGTTGTTCGGAATTACTGGGCGTAAAGCGCACGTAGGCGGCTTTGTAAGTTAGAGGTGAAAGCCC
GGGGCTCAACTCCGGAATTGCCTTTAAGACTGCATCGCTAGAATTGTGGAGAGGTAAGTGGAATTCCGAGTGTAGAGGTG
AAATTCGTAGATATTCGGAAGAACACCAGTGGCGAAGGCGACTTACTGGACACATATTGACGCTGAGGTGCGAAAGCGTG
GGGAGCAAACAGGATTAGATACCCTGGTAGTCCACGCCGTAAACGATGATGACTAGCTGTCCGGGCACTTAGTGCTTGGG
TGGCGCAGCTAACGCGTTAAGTCATCCGCCTGGGGAGTACGGCCGCAAGGTTA
>S026_79;size=35877;
TACGAAGGGGGCTAGCGTTGCTCGGAATCACTGGGCGTAAAGGGTGCGTAGGCGGGTCTTTAAGTCAGGGGTGAAATCCT
GGAGCTCAACTCCAGAACTGCCTTTGATACTGAAGATCTTGAGTTCGGGAGAGGTGAGTGGAACTGCGAGTGTAGAGGTG
AAATTCGTAGATATTCGCAAGAACACCAGTGGCGAAGGCGGCTCACTGGCCCGATACTGACGCTGAGGCACGAAAGCGTG


In [19]:
# This is the clustering command.
# Default is 97% minimum ID.
# Not recommended to use more than 97%.
# Creates the centroids, or "seeds"
# Then you can take them out
!usearch -cluster_otus data/finalQC.unique_sorted.fasta -otus data/otus.fasta

usearch v7.0.1090_i86linux32, 4.0Gb RAM (132Gb total), 40 cores
(C) Copyright 2013 Robert C. Edgar, all rights reserved.
http://drive5.com/usearch

Licensed to: chuck.peperanney@gmail.com

05:17 218Mb  100.0% 8828 OTUs
                             
Input seqs  581709 (581.7k)
      OTUs  8828
   Members  442908 (442.9k)
  Chimeras  129973 (130.0k)
   Max mem  218Mb
      Time  05:17
Throughput  1835.0 seqs/sec.



In [1]:
# Making another file
# Figure this out (what is this?)
# This is a script (fasta_number.py) that replaces fasta names with XXX1, XXX2, etc.
# In our case, it is replacing the names with OTU.1, OTU.2, etc., and outputs it into a file called otusn.fasta
!/opt/bioinfo/edgar_python_scripts/fasta_number.py data/otus.fasta OTU. > data/otusn.fasta

/bin/sh: 1: cannot create data/otusn.fasta: Directory nonexistent
/bin/sh: 1: fasta_number.py: not found


In [25]:
!head data/otusn.fasta

>OTU.1
TACGGAGGgggCTAGCGTTGTTCGGAATTACTGGGCGTAAAGCGCACGTAGGCGGCTTTGTAAGTTAGAGGTGAAAGCCC
GGGGCTCAACTCCGGAATTGCCTTTAAGACTGCATCGCTAGAATTGTGGAGAGGTAAGTGGAATTCCGAGTGTAGAGGTG
AAATTCGTAGATATTCGGAAGAACACCAGTGGCGAAGGCGACTTACTGGACACATATTGACGCTGAGGTGCGAAAGCGTG
GGGAGCAAACAGGATTAGATACCCTGGTAGTCCACGCCGTAAACGATGATGACTAGCTGTCCGGGCACTTAGTGCTTGGG
TGGCGCAGCTAACGCGTTAAGTCATCCGCCTGGGGAGTACGGCCGCAAGGTTA
>OTU.2
TACGAAGGgggCTAGCGTTGCTCGGAATCACTGGGCGTAAAGGGTGCGTAGGCGGGTCTTTAAGTCAGGGGTGAAATCCT
GGAGCTCAACTCCAGAACTGCCTTTGATACTGAAGATCTTGAGTTCGGGAGAGGTGAGTGGAACTGCGAGTGTAGAGGTG
AAATTCGTAGATATTCGCAAGAACACCAGTGGCGAAGGCGGCTCACTGGCCCGATACTGACGCTGAGGCACGAAAGCGTG


### Removing Chloroplast, Eukaryal, and Archaeal sequences

In [3]:
# You do need to assign taxonomy in order to pull out the Euks., etc.
# I could cp these files from the server to wherever I need them.
# Input is your fasta file
# Output is a fasta with taxonomy assinged (still working with unique seqs)
# This didn't work first, because Chuck had to delete a "jobs" folder in the tmp directory.

!parallel_assign_taxonomy_uclust.py \
-r data/97_Silva_111_rep_set_no_ambig.fasta \
-t data/Silva_111_taxa_map_full.txt \
-O 10 \
-i data/otusn.fasta \
-o data/otusn_tax

SyntaxError: invalid syntax (<ipython-input-3-df77c5e7b58d>, line 9)

In [31]:
# This makes a file of what we want to remove
# Could change this to pull out different groups.
# These primers actually had good Archaeal targets - so, it would be okay to include them.
!egrep "Chloroplast|Eukaryota|Archaea|Unassigned|mitochondria" \
data/otusn_tax/otusn_tax_assignments.txt \
| awk '{print $1}' > data/to_remove_tax.accnos

In [32]:
# wc is number of lines of the taxa that will be removed
!wc -l data/to_remove_tax.accnos

1058 data/to_remove_tax.accnos


In [33]:
# Looking at what you're removing
!head data/to_remove_tax.accnos

OTU.699
OTU.272
OTU.335
OTU.595
OTU.115
OTU.392
OTU.850
OTU.327
OTU.325
OTU.690


In [39]:
%%bash
# Remove.seqs command will actually remove these taxa
mothur "#remove.seqs(fasta=data/otusn.fasta, \
accnos=data/to_remove_tax.accnos)" #> /dev/null







mothur v.1.32.1
Last updated: 10/16/2013

by
Patrick D. Schloss

Department of Microbiology & Immunology
University of Michigan
pschloss@umich.edu
http://www.mothur.org

When using, please cite:
Schloss, P.D., et al., Introducing mothur: Open-source, platform-independent, community-supported software for describing and comparing microbial communities. Appl Environ Microbiol, 2009. 75(23):7537-41.

Distributed under the GNU General Public License

Type 'help()' for information on the commands that are available

Type 'quit()' to exit program



mothur > remove.seqs(fasta=data/otusn.fasta, accnos=data/to_remove_tax.accnos)
Removed 1058 sequences from your fasta file.

Output File Names: 
data/otusn.pick.fasta


mothur > quit()


### Mapping Reads

In [40]:
# Pulling out the sample identifier.
# It is adding a portion to the finalQC file that has the barcode label.
# Then we can use this later
# Now we will see how these reads map to the defined centroids (after removing EuK, etc.)
# Basically, we cut, cut, refined our fasta to make our OTU centroids.
# THEN, we went back to our original QC'd total fasta file and will throw it all against these nicely defined seeds.
# Anything that doesn't match, we won't keep.
!awk -F"_" \
'BEGIN{OFS=";"}{ if ( substr($1,0,1) == ">"){ print $0,"barcodelabel=",$1 } else { print $0 } }' \
data/finalQC.fasta | \
sed 's/;>//' > data/finalQC_usearchfmt.fasta

In [41]:
!head data/finalQC_usearchfmt.fasta

>S133_53;barcodelabel=S133
TACGTAGGGGGCAAGCGTTGTCCGGAATTATTGGGCGTAAAGGGCGCGTAGGCGGTTCGGTAAGTCACCTGTGAAACCTCTGGGCTCAACTCAGGGCCTGCAGGCGAAACTGCCGTGCTGGAGTGTGGGAGAGGTGCGTGGAATTCCCGGTGTAGCGGTGAAATGCGTAGATATCGGGAGGAACACCTGTGGCGAAAGCGGCGCACTGGACCACAACTGACGCTGAGGCGCGAAAGCTAGGGGAGCAAACAGGATTAGATACCCTGGTAGTCCTAGCCCTAAACGATGGATGCTTGGTGTGTCGGGTACCCAATCCCGACGTGCCGAAGCTAACGCGATAAGCATCCCGCCTGGGGAGTACGGTCGCAAGGCTG
>S009_54;barcodelabel=S009
TACGAAGGTGGCAAGCGTTATTCGGATTTACTGGGCGTACAGGGAGCGTAGGCGGTTCGGTAAGCCCTTTGTGAAATCTTCAGGCTTAACCTGGAACAGTCGGAGGGGACTGCCGGGCTAGAGGACGGGAGAGGAGCGCGGAATTCCCGGTGTAGCGGTGAAATGCGTAGAGATCGGGAGGAAGGCCGGTGGCGAAGGCGGCGCTCTGGAACGTTTCTGATGCTGAGGCTCGAAAGCGTGGGGAGCAAACAGGATTAGATACCCTGGTAGTCCACGCCTTAAACTATGAATACTAAGTGTCGGCGGGTTACCGCCGGTGCCGCAGCTAACGCATTAAGTATTCCGCCTGGGAAGTACGGCCGCAAGGTTG
>S026_79;barcodelabel=S026
TACGAAGGGGGCTAGCGTTGCTCGGAATCACTGGGCGTAAAGGGTGCGTAGGCGGGTCTTTAAGTCAGGGGTGAAATCCTGGAGCTCAACTCCAGAACTGCCTTTGATACTGAAGATCTTGAGTTCGGGAGAGGTGAGTGGAACTGCGAGTGTAGAGGTGAAATTCGTAGATA

In [44]:
# This is where the actual OTUs are being assigned. We choose 97% sequence ID threshold here.
# This might take a while - like 5 minutes
# Depending ont he clustering algorithm, like pairwise... it would take, like, days on the same number of processors.
# This is why usearch (centroid-based) is so much better
# But is it more biologically relevant? ... maybe, maybe not.
# Edgar is showing it's not that bad.

# We take our total QC data (modified above to have the sample ID extracted)
# We compare it to the otusn.pick.fasta database we made above
# We produce a readmap.uc file which tells us how the reads from our finalQC file map to the otusn seed database.

!usearch -usearch_global data/finalQC_usearchfmt.fasta \
-db data/otusn.pick.fasta \
-strand plus -id 0.97 \
-uc data/readmap.uc \
-threads 15

usearch v7.0.1090_i86linux32, 4.0Gb RAM (132Gb total), 40 cores
(C) Copyright 2013 Robert C. Edgar, all rights reserved.
http://drive5.com/usearch

Licensed to: chuck.peperanney@gmail.com

00:00  19Mb Reading data/otusn.pick.fasta, 3.0Mb
00:00  22Mb 7770 seqs, min 370, avg 373, max 376nt
00:00  22Mb  100.0% Masking
00:00  22Mb  100.0% Word stats
00:00  34Mb  100.0% Building slots
00:00  34Mb  100.0% Build index
02:18 189Mb  100.0% Searching, 81.4% matched


In [4]:
# Makes an OTU table
# It will tell me the OTU ID, and then for all the samples, which OTUs it has sequences from.
!python /opt/bioinfo/edgar_python_scripts/uc2otutab.py data/readmap.uc > data/otu_table.txt

/bin/sh: 1: cannot create data/otu_table.txt: Directory nonexistent


In [5]:
# Issues with biom table formatting
!if [ -f data/otu_table.biom ]; then rm data/otu_table.biom; fi #This is to mitigate a biom bug
!biom convert -i data/otu_table.txt -o data/otu_table.biom --table-type"otu table"

Usage: biom convert [options] {-i/--input-fp INPUT-FP -o/--output-fp OUTPUT-FP}

[] indicates optional input (order unimportant)
{} indicates required input (order unimportant)

Convert between BIOM and 'classic' (tab-delimited) table formats. Detailed usage examples can be found here: http://biom-format.org/documentation/biom_conversion.html

Example usage: 
Print help message and exit
 biom convert -h

Converting from classic to BIOM format: Convert the classic file table.txt to a sparse BIOM format OTU table
 biom convert -i table.txt -o table.biom --table-type "otu table"

biom convert: error: option -i: file does not exist: 'data/otu_table.txt'


In [47]:
# Issues with biom table formatting
!if [ -f data/otu_table_summary.txt ]; then rm data/otu_table_summary.txt; fi #This is to mitigate a biom bug
!biom summarize-table -i data/otu_table.biom -o data/otu_table_summary.txt

In [48]:
# This tells us the overall data info
# Num obs = OTUs
# total count = total seqs
# Chantal had 50% reduction after QC.

!cat data/otu_table_summary.txt

# Sample 211 was just bad 8 sequesce. Final day, plot 11, 
# Other samples have 8,000 while biggest one has 194,000 sequences.
# 8M total sequences

Num samples: 120
Num observations: 7770
Total count: 8330257
Table density (fraction of non-zero values): 0.412
Table md5 (unzipped): db748eb39a90836c39c72248bfadbc13

Counts/sample summary:
 Min: 8.0
 Max: 194356.0
 Median: 53598.500
 Mean: 69418.808
 Std. dev.: 44070.005
 Sample Metadata Categories: None provided
 Observation Metadata Categories: None provided

Counts/sample detail:
 S211: 8.0
 S038: 8830.0
 S242: 13813.0
 S039: 15038.0
 S129: 20512.0
 S245: 21660.0
 S029: 23329.0
 S021: 25060.0
 S239: 25681.0
 S037: 26554.0
 S017: 26555.0
 S014: 26968.0
 S027: 27587.0
 S013: 27713.0
 S018: 28479.0
 S243: 28671.0
 S012: 31182.0
 S131: 31203.0
 S145: 31203.0
 S042: 31996.0
 S246: 32120.0
 S114: 32298.0
 S044: 32861.0
 S016: 34523.0
 S148: 34617.0
 S006: 35309.0
 S247: 35635.0
 S036: 36097.0
 S043: 36098.0
 S023: 36682.0
 S003: 36728.0
 S033: 37283.0
 S034: 37348.0
 S244: 37578.0
 S007: 37877.0
 S233: 39284.0
 S028: 39352.0
 S004: 39569.0
 S022: 39898.0
 S137: 39955.0
 S048: 40087.0
 S

In [14]:
# My final data will be as follows:
# My OTU table
# This has OTUs and counts per sample
otu_table.biom
# This file has the sequences of my OTUs
otusn.pick.fasta
# These should be the only files I care about from now on.
# Also note, I have created an aligned file
ssu-aln.bacteria.mask.fasta
# And I have also a tree
ssu-aln.bacteria.mask.tre